In [4]:
%matplotlib inline

##TODO postprocessing and filtering

In [2]:
import os,sys,glob,shutil
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from pygeotools.lib import iolib,geolib,malib,warplib,timelib
from imview import pltlib
from itertools import combinations
from distutils.spawn import find_executable

In [6]:
%cd /nobackup/sbhusha1/oic_challenge/la_valette_landslide/Data-2_SLIDE-LaValette/Sentinel-2/

/nobackupp11/sbhusha1/oic_challenge/la_valette_landslide/Data-2_SLIDE-LaValette/Sentinel-2


## S2 MGM

In [19]:
' '.join(date_list)

'20150829T103026 20160902T103022 20170907T103021 20180828T103019 20190803T103029 20200906T102559 20210827T103021 20220901T102611 20230906T102601'

In [18]:
date_list = ['20150829T103026', '20160902T103022', '20170907T103021','20180828T103019', '20190803T103029', '20200906T102559', '20210827T103021', '20220901T102611', '20230906T102601']
img_list = []
for idx,date in  enumerate(date_list):
    img = sorted(glob.glob(f'T31TGK_{date}*.tif'))[0]
    img_list.append(img)
comb = list(combinations(img_list,2))
job_fn = '/nobackupp11/sbhusha1/oic_challenge/la_valette_landslide/results/s2_mgm/s2_jobs.txt'
with open(job_fn,'w') as f:
    for pairs in comb:
        img1,img2 = pairs
        cmd = '/swbuild/sbhusha1/pip_git_sw/debris_cover_smb/scripts/disp_mgm_corr.py'
        parameter_str = f" -dt px -skip_rate 1 -pleiades {os.path.abspath(img1)} {os.path.abspath(img2)}\n"
        f.write(cmd+parameter_str)

## S2 Block

In [10]:
date_list = ['20150829T103026', '20160902T103022', '20170907T103021','20180828T103019', '20190803T103029', '20200906T102559', '20210827T103021', '20220901T102611', '20230906T102601']
img_list = []
for idx,date in  enumerate(date_list):
    img = sorted(glob.glob(f'T31TGK_{date}*.tif'))[0]
    img_list.append(img)
comb = list(combinations(img_list,2))
job_fn = '/nobackupp11/sbhusha1/oic_challenge/la_valette_landslide/results/s2_block/s2_jobs.txt'
with open(job_fn,'w') as f:
    for pairs in comb:
        img1,img2 = pairs
        cmd = find_executable('vmap.py')
        parameter_str = f" {os.path.abspath(img1)} {os.path.abspath(img2)} -dt none\n"
        f.write(cmd+parameter_str)

## Plieades

In [11]:
%cd /nobackup/sbhusha1/oic_challenge/la_valette_landslide/Data-2_SLIDE-LaValette/Pleiades/Pleiades_ortho

/nobackupp11/sbhusha1/oic_challenge/la_valette_landslide/Data-2_SLIDE-LaValette/Pleiades/Pleiades_ortho


In [14]:
ls

20140620_Pleiades_LaValette_ortho_crop.tif
20190608_Pleiades_LaValette_ortho_crop.tif
20220801_Pleiades_LaValette_ortho_crop.tif


In [13]:
date_list = ['20140620','20190608','20220801']
img_list = []
for idx,date in  enumerate(date_list):
    img = sorted(glob.glob(f'{date}*.tif'))[0]
    img_list.append(img)
comb = list(combinations(img_list,2))
job_fn = '/nobackupp11/sbhusha1/oic_challenge/la_valette_landslide/results/phr_ortho/phr_ortho_jobs.txt'
with open(job_fn,'w') as f:
    for pairs in comb:
        img1,img2 = pairs
        cmd = '/swbuild/sbhusha1/pip_git_sw/debris_cover_smb/scripts/disp_mgm_corr.py'
        parameter_str = f" -dt px -skip_rate 1 -pleiades {os.path.abspath(img1)} {os.path.abspath(img2)}\n"
        f.write(cmd+parameter_str)

## Prepare displacement maps in m

In [7]:
%cd /nobackup/sbhusha1/oic_challenge/la_valette_landslide/results/

/nobackupp11/sbhusha1/oic_challenge/la_valette_landslide/results


In [38]:
disp_rd_fn_list = sorted(glob.glob('s2_block/T*/T*F.tif'))
print(f"The program produced {len(disp_rd_fn_list)} velocity pairs")
                         

The program produced 36 velocity pairs


In [12]:
outdir = 'asp_block/Sentinel2/'
ic_of_outdir = os.path.join(outdir,'IC-OF_RESULTS')
velocity_outdir = os.path.join(outdir,'VELOCITY')
displacement_outdir = os.path.join(outdir,'DISPLACEMENT')
if not os.path.exists(outdir):
    os.makedirs(outdir)
if not os.path.exists(ic_of_outdir):
    os.makedirs(ic_of_outdir)
if not os.path.exists(velocity_outdir):
    os.makedirs(velocity_outdir)
if not os.path.exists(displacement_outdir):
    os.makedirs(displacement_outdir)

In [13]:
sample_disp_fn = disp_rd_fn_list[0]

In [14]:
sample_disp_fn

's2_block/T31TGK_20150829T103026_B04__T31TGK_20160902T103022_B04_vmap_minm_35px_spm1/T31TGK_20150829T103026_B04__T31TGK_20160902T103022_B04_vmap_minm_35px_spm1-F.tif'

In [16]:
ds = iolib.fn_getds(sample_disp_fn)
disp_x,disp_y = [iolib.ds_getma(ds,x) for x in [1,2]]

In [27]:
res_x,res_y = geolib.get_res(ds)

In [31]:
ds.GetGeoTransform()

(788220.0, 10.0, 0.0, 4925430.0, 0.0, -10.0)

In [34]:
def write_out_oic_results_block_matching(disp_fn):
    t1, t2 = timelib.fn_getdatetime_list(disp_fn)
    ds = iolib.fn_getds(disp_fn)
    disp_x,disp_y = [iolib.ds_getma(ds,x) for x in [1,2]]
    t1_str = t1.strftime('%Y%m%d') 
    t2_str = t2.strftime('%Y%m%d') 
    dt = t2 - t1
    t_factor = timelib.get_t_factor(t1,t2)
    gt = ds.GetGeoTransform()
    proj = ds.GetProjection()
    ic_oc_vx_fn = os.path.join(ic_of_outdir,f'EW_{t1_str}-{t2_str}_raw.tif')
    ic_oc_vy_fn = os.path.join(ic_of_outdir,f'NS_{t1_str}-{t2_str}_raw.tif')
    iolib.writeGTiff(disp_x, ic_oc_vx_fn,ndv=-9999,gt=gt,proj=proj)
    iolib.writeGTiff(disp_y, ic_oc_vy_fn,ndv=-9999,gt=gt,proj=proj)
    
    disp_oc_vx_fn = os.path.join(displacement_outdir,f'EW_{t1_str}-{t2_str}_disp_tot.tif')
    disp_oc_vy_fn = os.path.join(displacement_outdir,f'NS_{t1_str}-{t2_str}_disp_tot.tif')
    res_x,res_y = geolib.get_res(ds)
    iolib.writeGTiff(disp_x*res_x, disp_oc_vx_fn,ndv=-9999,gt=gt,proj=proj)
    iolib.writeGTiff(disp_y*res_y, disp_oc_vy_fn,ndv=-9999,gt=gt,proj=proj)
    
    vel_oc_vx_fn = os.path.join(velocity_outdir,f'EW_{t1_str}-{t2_str}_vel_mean.tif')
    vel_oc_vy_fn = os.path.join(velocity_outdir,f'NS_{t1_str}-{t2_str}_vel_mean.tif')
    iolib.writeGTiff((disp_x*res_x)/t_factor, vel_oc_vx_fn,ndv=-9999,gt=gt,proj=proj)
    iolib.writeGTiff((disp_y*res_y)/t_factor, vel_oc_vy_fn,ndv=-9999,gt=gt,proj=proj)

In [35]:
write_out_oic_results_block_matching(sample_disp_fn)


In [39]:
for disp in disp_rd_fn_list:
    write_out_oic_results_block_matching(disp)

## MGM writeout

In [40]:
outdir = 'asp_mgm/Sentinel2/'
ic_of_outdir = os.path.join(outdir,'IC-OF_RESULTS')
velocity_outdir = os.path.join(outdir,'VELOCITY')
displacement_outdir = os.path.join(outdir,'DISPLACEMENT')
if not os.path.exists(outdir):
    os.makedirs(outdir)
if not os.path.exists(ic_of_outdir):
    os.makedirs(ic_of_outdir)
if not os.path.exists(velocity_outdir):
    os.makedirs(velocity_outdir)
if not os.path.exists(displacement_outdir):
    os.makedirs(displacement_outdir)

In [41]:
vx_fn_list = sorted(glob.glob('s2_mgm/T*/T*_vx*.tif'))
vy_fn_list = sorted(glob.glob('s2_mgm/T*/T*_vy*.tif'))
print(f"The program produced {len(vx_fn_list)} velocity pairs")
                         

The program produced 36 velocity pairs


In [43]:
def write_out_oic_results_mgm_matching(vx_fn,vy_fn):
    t1, t2 = timelib.fn_getdatetime_list(vx_fn)
    vx_ds = iolib.fn_getds(vx_fn)
    vy_ds = iolib.fn_getds(vy_fn)
    disp_x,disp_y = [iolib.ds_getma(ds) for ds in [vx_ds,vy_ds]]
    t1_str = t1.strftime('%Y%m%d') 
    t2_str = t2.strftime('%Y%m%d') 
    dt = t2 - t1
    t_factor = timelib.get_t_factor(t1,t2)
    gt = vx_ds.GetGeoTransform()
    proj = vx_ds.GetProjection()
    ic_oc_vx_fn = os.path.join(ic_of_outdir,f'EW_{t1_str}-{t2_str}_raw.tif')
    ic_oc_vy_fn = os.path.join(ic_of_outdir,f'NS_{t1_str}-{t2_str}_raw.tif')
    iolib.writeGTiff(disp_x, ic_oc_vx_fn,ndv=-9999,gt=gt,proj=proj)
    iolib.writeGTiff(disp_y, ic_oc_vy_fn,ndv=-9999,gt=gt,proj=proj)
    
    disp_oc_vx_fn = os.path.join(displacement_outdir,f'EW_{t1_str}-{t2_str}_disp_tot.tif')
    disp_oc_vy_fn = os.path.join(displacement_outdir,f'NS_{t1_str}-{t2_str}_disp_tot.tif')
    res_x,res_y = geolib.get_res(vx_ds)
    iolib.writeGTiff(disp_x*res_x, disp_oc_vx_fn,ndv=-9999,gt=gt,proj=proj)
    iolib.writeGTiff(disp_y*res_y, disp_oc_vy_fn,ndv=-9999,gt=gt,proj=proj)
    
    vel_oc_vx_fn = os.path.join(velocity_outdir,f'EW_{t1_str}-{t2_str}_vel_mean.tif')
    vel_oc_vy_fn = os.path.join(velocity_outdir,f'NS_{t1_str}-{t2_str}_vel_mean.tif')
    iolib.writeGTiff((disp_x*res_x)/t_factor, vel_oc_vx_fn,ndv=-9999,gt=gt,proj=proj)
    iolib.writeGTiff((disp_y*res_y)/t_factor, vel_oc_vy_fn,ndv=-9999,gt=gt,proj=proj)

In [44]:
for idx,vx_fn in enumerate(vx_fn_list):
    write_out_oic_results_mgm_matching(vx_fn,vy_fn_list[idx])

### PHR results

In [45]:
outdir = 'asp_mgm/PHR_Ortho/'
ic_of_outdir = os.path.join(outdir,'IC-OF_RESULTS')
velocity_outdir = os.path.join(outdir,'VELOCITY')
displacement_outdir = os.path.join(outdir,'DISPLACEMENT')
if not os.path.exists(outdir):
    os.makedirs(outdir)
if not os.path.exists(ic_of_outdir):
    os.makedirs(ic_of_outdir)
if not os.path.exists(velocity_outdir):
    os.makedirs(velocity_outdir)
if not os.path.exists(displacement_outdir):
    os.makedirs(displacement_outdir)

In [46]:
vx_fn_list = sorted(glob.glob('phr_ortho/2*/2*_vx*.tif'))
vy_fn_list = sorted(glob.glob('phr_ortho/2*/2*_vy*.tif'))
print(f"The program produced {len(vx_fn_list)} velocity pairs")

The program produced 3 velocity pairs


In [47]:
for idx,vx_fn in enumerate(vx_fn_list):
    write_out_oic_results_mgm_matching(vx_fn,vy_fn_list[idx])

## Format Quinn's results

In [1]:
%cd /nobackup/sbhusha1/oic_challenge/

/nobackupp11/sbhusha1/oic_challenge


In [3]:
displacement_results = sorted(glob.glob('la_valette_landslide/results/quinn/OIC_la_valette_landslide/Sentinel2/DISPLACEMENT/*ndv.tif'))
velocity_results = sorted(glob.glob('la_valette_landslide/results/quinn/OIC_la_valette_landslide/Sentinel2/VELOCITY/*ndv.tif'))
px_results = sorted(glob.glob('la_valette_landslide/results/quinn/OIC_la_valette_landslide/Sentinel2/IC-OF_RESULTS/*ndv.tif'))

In [5]:
len(velocity_results)

72

In [7]:
for idx,displacement in enumerate(displacement_results):
    displacement_outdir = 'university_of_washington_results/Site2/Sentinel2/autoRIFT/DISPLACEMENT'
    velocity_outdir = 'university_of_washington_results/Site2/Sentinel2/autoRIFT/VELOCITY'
    px_outdir = 'university_of_washington_results/Site2/Sentinel2/autoRIFT/IC-OF_RESULTS'
    if not os.path.exists(displacement_outdir):
        os.makedirs(displacement_outdir)
    if not os.path.exists(velocity_outdir):
        os.makedirs(velocity_outdir)
    if not os.path.exists(px_outdir):
        os.makedirs(px_outdir)
    out_displacement = os.path.join(displacement_outdir,os.path.basename(displacement).split('_ndv.tif')[0]+'.tif')
    out_velocity = os.path.join(velocity_outdir,os.path.basename(velocity_results[idx]).split('_ndv.tif')[0]+'.tif')
    out_px = os.path.join(px_outdir,os.path.basename(px_results[idx]).split('_ndv.tif')[0]+'.tif')
    shutil.copy2(displacement,out_displacement)
    shutil.copy2(velocity_results[idx],out_velocity)
    shutil.copy2(px_results[idx],out_px)